# LSTM Autoencoder
* Used for feature extractor to feed next model that has fixed length of input.

## Motivation
* Training data using human intervention for feature I believe is not something scalable. If we look into image, initially most classification with high accuracy came from feature built by hand or from expert intuition but look at now mostly deep learning outperform traditional machine learning at image classification
* To have ability to learn from full example of dataset 

## TODO
* Use variable length of input without padding and truncating

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import h5py
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("safety"))

# Any results you write to the current directory are saved as output.

# lstm autoencoder recreate sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import backend as K
from numpy import array
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.models import load_model
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm
import re

['.DS_Store', 'labels', 'data_dictionary.xlsx', 'features']


Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 689382141307509918
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11286285517
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13583097981217109386
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 7} ) 
sess = tf.Session(config=config) 
K.set_session(sess)

In [6]:
#booking_ids = np.load('booking_ids.npy')
#Xpad = np.load('Xpad.npy')
# read back
with h5py.File('data/Xpad.h5', 'r') as hf:
    Xpad = hf['Xpad'][:]

In [7]:
# Taken from https://stackoverflow.com/a/55264099
def FindBatchSize(model):
    """#model: model architecture, that is yet to be trained"""
    import os, sys, psutil, gc, tensorflow, keras
    import numpy as np
    from keras import backend as K
    BatchFound= 16

    try:
        total_params= int(model.count_params());    GCPU= "CPU"
        #find whether gpu is available
        try:
            if K.tensorflow_backend._get_available_gpus()== []:
                GCPU= "CPU";    #CPU and Cuda9GPU
            else:
                GCPU= "GPU"
        except:
            from tensorflow.python.client import device_lib;    #Cuda8GPU
            def get_available_gpus():
                local_device_protos= device_lib.list_local_devices()
                return [x.name for x in local_device_protos if x.device_type == 'GPU']
            if "gpu" not in str(get_available_gpus()).lower():
                GCPU= "CPU"
            else:
                GCPU= "GPU"

        #decide batch size on the basis of GPU availability and model complexity
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <1000000):
            BatchFound= 64    
        if (os.cpu_count() <16) and (total_params <500000):
            BatchFound= 64  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <2000000) and (total_params >=1000000):
            BatchFound= 32      
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=2000000) and (total_params <10000000):
            BatchFound= 16  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=10000000):
            BatchFound= 8       
        if (os.cpu_count() <16) and (total_params >5000000):
            BatchFound= 8    
        if total_params >100000000:
            BatchFound= 1

    except:
        pass
    try:

        #find percentage of memory used
        memoryused= psutil.virtual_memory()
        memoryused= float(str(memoryused).replace(" ", "").split("percent=")[1].split(",")[0])
        if memoryused >75.0:
            BatchFound= 8
        if memoryused >85.0:
            BatchFound= 4
        if memoryused >90.0:
            BatchFound= 2
        if total_params >100000000:
            BatchFound= 1
        print("Batch Size:  "+ str(BatchFound));    gc.collect()
    except:
        pass

    memoryused= [];    total_params= [];    GCPU= "";
    del memoryused, total_params, GCPU;    gc.collect()
    return BatchFound

In [13]:
# padding value
pad_value = 2
max_seq_len = Xpad.shape[1]
dimension = Xpad.shape[2]

In [35]:
# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    ModelCheckpoint(
        filepath='models/lstm-autoencoder-best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True, save_weights_only=False),
    EarlyStopping(monitor='val_loss', patience=1, mode='min')
]


In [36]:
# autoencoder
# A bit messy here, tuning the hyperparamter by hand for now
# define model
model = Sequential()
model.add(Masking(mask_value=pad_value, input_shape=(max_seq_len, dimension)))
#model.add(LSTM(100, activation='relu', input_shape=(max_seq_len,dimension), return_sequences=False))
#model.add(LSTM(256, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=False))
model.add(RepeatVector(max_seq_len))
#model.add(LSTM(256, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(dimension)))
opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1.)
#opt = optimizers.Adam(lr=0.01, clipnorm=1.)
model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mse'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9 (Masking)          (None, 1000, 11)          0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 100)               44800     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 1000, 100)         0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 1000, 100)         80400     
_________________________________________________________________
time_distributed_11 (TimeDis (None, 1000, 11)          1111      
Total params: 126,311
Trainable params: 126,311
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
batch_size = FindBatchSize(model)

Batch Size:  64


In [38]:
Xpad.shape

(19863, 1000, 11)

In [39]:
%%time
# fit model
# used bigger batch size and small epochs due to time constraint
# encountered NaN issue has been debugging for hours, ref https://towardsdatascience.com/debugging-a-machine-learning-model-written-in-tensorflow-and-keras-f514008ce736
model.fit(Xpad, Xpad, epochs=10, verbose=1, batch_size=1000, callbacks=callbacks_list,
                      validation_split=0.2)

Train on 15890 samples, validate on 3973 samples
Epoch 1/10
15890/15890 [==============================] - 105s 7ms/step - loss: 1.0251 - mean_squared_error: 1.6987 - val_loss: 0.9416 - val_mean_squared_error: 1.5160
Epoch 2/10
15890/15890 [==============================] - 102s 6ms/step - loss: 0.8337 - mean_squared_error: 1.2921 - val_loss: 0.7241 - val_mean_squared_error: 1.0103
Epoch 3/10
15890/15890 [==============================] - 102s 6ms/step - loss: 0.7235 - mean_squared_error: 0.8956 - val_loss: 0.7233 - val_mean_squared_error: 0.8672
Epoch 4/10
15890/15890 [==============================] - 102s 6ms/step - loss: 0.7168 - mean_squared_error: 0.9115 - val_loss: 0.7140 - val_mean_squared_error: 0.9643
Epoch 5/10
15890/15890 [==============================] - 101s 6ms/step - loss: 0.7120 - mean_squared_error: 0.9746 - val_loss: 0.7124 - val_mean_squared_error: 0.9792
Epoch 6/10
15890/15890 [==============================] - 100s 6ms/step - loss: 0.7111 - mean_squared_error: 0.

In [45]:
# Retrieve best model
def get_best_model(path, file_prefix="", mode='min'):
  files = os.listdir(path)
  data = []
  best = 0
  for i,file in enumerate(files):
    r = re.search(file_prefix+"\.(\d+?)-([\d.]+?)\.h5", file, re.IGNORECASE)
    if r:
        data.append({'file':os.path.join(path, file), 'epoch': r.group(1), 'val':r.group(2)})
        if mode == 'min' and data[best]['val'] > data[-1]['val']:
          best = i
        elif mode == 'max' and data[best]['val'] < data[-1]['val']:
          best = i
    return data[best]
            
  
best_model = get_best_model("models", "lstm-autoencoder-best_model")
model1 = load_model(best_model['file'])
print(best_model)

{'val': '0.71', 'epoch': '04', 'file': 'models/lstm-autoencoder-best_model.04-0.71.h5'}


In [48]:
%%time
# connect the encoder LSTM as the output layer
model2 = Model(inputs=model1.inputs, outputs=model1.layers[1].output)
#plot_model(model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
print(model2.summary())
yhat = model2.predict(Xpad)
print(yhat.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9_input (InputLayer) (None, 1000, 11)          0         
_________________________________________________________________
masking_9 (Masking)          (None, 1000, 11)          0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 100)               44800     
Total params: 44,800
Trainable params: 44,800
Non-trainable params: 0
_________________________________________________________________
None
(19863, 100)
CPU times: user 18min 13s, sys: 4min 8s, total: 22min 22s
Wall time: 9min 5s


In [47]:
%%time
# predict single booking id
model2.predict(Xpad[0].reshape(1,1000,11))

CPU times: user 2.18 s, sys: 351 ms, total: 2.53 s
Wall time: 1.36 s


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.7156408e-02,
        8.0527468e-03, 4.7768103e-03, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 7.1555769e-06, 2.9005981e-03, 1.6591525e-01,
        9.1693051e-02, 6.0353428e-02, 6.8075177e-12, 2.4456780e-02,
        8.9798167e-02, 3.8826701e-04, 0.0000000e+00, 1.1437282e-03,
        5.5426961e-01, 1.1586640e-01, 2.5789881e-01, 1.4813118e-01,
        0.0000000e+00, 6.3901901e-02, 1.4552786e-03, 4.0595583e-03,
        2.8994054e-02, 0.0000000e+00, 4.2283259e-02, 1.1469223e-02,
        5.2422974e-02, 1.6570378e-02, 2.9908579e-06, 2.1865903e-14,
        5.5143732e-07, 3.0423421e-02, 0.0000000e+00, 1.6376851e-03,
        1.1907379e-01, 2.5800967e-01, 6.9169626e-02, 3.4278724e-03,
        2.4409366e-01, 1.1782951e-01, 4.6075750e-02, 2.2797266e-01,
        9.4397262e-02, 3.3995082e-05, 1.0834193e-03, 1.7993878e-01,
        1.3978146e-01, 3.5642271e-04, 2.4428770e-01, 0.0000000e+00,
        1.4044857e-01, 4.2119850e-03, 1.2152109e

In [49]:
cols = []
for i in range(100):
  cols.append("feat_"+str(i))

In [50]:
# this can be used in next model
pd.DataFrame(yhat, columns=cols).to_parquet("data/features.parquet")